### Importação das bibliotecas 

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from unidecode import unidecode
from datetime import datetime
import re
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

### Leitura do arquivo e conversão para o formato parquet


#### Leitura com Pandas

In [ ]:
# Leitura do arvuivo em CSV
emissao_co2 = pd.read_csv("Emissão_CO2_por_países.csv")

# Escrita em Parquet com compressão Snappy (mais comum). Para arquivos pequenos e execução local, é uma alternativa boa.
# Para ambientes distribuidos e arquivos grandes, é mais adequado a utilização do PySpark
emissao_co2.to_parquet("emissao_co2.parquet", engine="pyarrow", compression="snappy")

#### Leitura com PySpark

In [ ]:

""" # Essa etapa foi apresentada para teste
# Criar uma SparkSession
spark = SparkSession.builder.appName("CSV to Parquet").getOrCreate()

# Caminho para o arquivo CSV de entrada
csv_path = "Emissão_CO2_por_países.csv"

# Caminho para o diretório de saída Parquet
# No PySpark, cria-se uma pasta contendo vários arquivos
parquet_path = "emissao_co2.parquet"

# Leitura do CSV
df = spark.read.option("header", "true").csv(csv_path)

# Escrita em formato Parquet
df.write.mode("overwrite").parquet(parquet_path)

# Encerrar a SparkSession (opcional)
spark.stop()
"""

In [ ]:
# Leitura do arquivo em parquet
emissao_co2 = pd.read_parquet("emissao_co2.parquet")

In [ ]:
emissao_co2.head()

In [ ]:
emissao_co2.tail()

In [ ]:
# Obtendo informações gerais sobre a base de dados
emissao_co2.info()

In [ ]:
# Verificando valores faltantes
emissao_co2.isnull().sum()

In [ ]:
# A base de dados conta com um total de 191 países, sendo que existem occorências vazias
len(set(emissao_co2['Country']))

In [ ]:
# Verificação de ocorrências vazias para Países
sum(emissao_co2['Country'].isnull())

In [ ]:
# Verificação de ocorrências vazias para Países
sum(emissao_co2['Region'].isnull())

In [ ]:
# As regiões apresentam variações em suas nomenclaturas
set(emissao_co2['Region'])

### Padronização das Regiões
##### Nessa seção, será padronizada as nomenclaturas de Regiões

In [ ]:
""" 
re.compile(): cria um padrão de expressão regular para buscar certos termos.

r"...": string "raw" (crua), onde \ não é tratado como caractere especial do Python.

\b: representa uma fronteira de palavra (para evitar que, por exemplo, "asiático" bata com "Asia").

re.IGNORECASE: faz a busca sem diferenciar maiúsculas de minúsculas.
"""
# Gera um padrão de busca para cada região 
formato_padrao = {
    "Asia": re.compile(r"\basia\b", re.IGNORECASE), 
    "Africa": re.compile(r"\bafrica\b|\bfric", re.IGNORECASE),
    "Europa": re.compile(r"\beuro|\beuropa", re.IGNORECASE),
    "Oceania": re.compile(r"\bocean|\boceania", re.IGNORECASE),
    "Americas": re.compile(r"\bameric|\bamerica|\baméric", re.IGNORECASE)
}

# Função para normalizar as variações das nomenclaturas de Regiões
def normalizar_regiao(regiao):
    if pd.isnull(regiao): # Verifica se a variável regiao está vazia ou nula (NaN).
        return "Outros"   # Se estiver, retorna "Outros" como valor padrão.
    # remove espaços extras antes e depois.
    # transforma tudo em letras minúsculas.
    # remove acentos e caracteres especiais.
    regiao_normalizada = unidecode(regiao.strip().lower()) 
    
    # Percorre cada item no dicionário formato_padrao
    # padrao.search(...) → verifica se o texto (já limpo e sem acento) bate com algum dos padrões definidos
    for nome_normalizado, padrao in formato_padrao.items():
        if padrao.search(regiao_normalizada):
            return nome_normalizado

    return "Outros"

In [ ]:
# Os 24 dados faltantes de Regiões listados anteriormente, são referentes ao país Zimbabwe. 
# Preenche os valores faltantes para o país Zimbabwe
emissao_co2.loc[(emissao_co2['Country'] == 'Zimbabwe') & 
                (emissao_co2['Region'].isnull()),'Region'] = 'Africa'

In [ ]:
""" 
Aplica a função normalizar_regiao linha por linha (ou seja, valor por valor da coluna).

Para cada valor da coluna "Region", a função vai:

Verificar se está nulo (NaN) → retorna "Outros"

Remover acentos, espaços extras, letras maiúsculas

Verificar se a string contém padrões como "afric", "euro", "americ", etc.

Retornar uma versão padronizada:

"Africa"

"Asia"

"Americas"

"Europa"

"Oceania"

Ou "Outros" se não encontrar correspondência

Trata acentos e variações de grafia

Padroniza os nomes de região

Facilita filtros, agrupamentos e visualizações futuras

Evita erros como "África" vs "africano" vs "africa"

"""

# Após preencher os dados faltantes de Região, vamos normalizar as variações de região aplicando a função normalizar_regiao
emissao_co2['Region'] = emissao_co2['Region'].apply(normalizar_regiao)



In [ ]:
# Regiões ajustadas
set(emissao_co2['Region'])

In [ ]:
emissao_co2.tail()

### Padronização das Datas
##### Conforme constatado anteriormente, o arquivo conta com datas despadronizadas, faltantes. Nesta seção, vamos manter apenas as datas com valores adequados. 
##### Outro detalhe, é que os dados para análise propriamente ditos contemplam apenas o período entre 1990 até 2019. Datas anteriores à 1990 serão desconsideradas.

In [ ]:
""" 
É uma função do pandas que converte strings em objetos do tipo datetime64, que permitem:

ordenar por data,

calcular intervalos de tempo,

extrair ano, mês, dia, etc.

Diz ao pandas o que fazer quando a string não puder ser convertida em data:

"coerce" → converte valores inválidos para NaT ("Not a Time", tipo nulo de data)

Exemplo: "31-02-2022" → inválido → vira NaT
"""

# Converte a coluna Date para datetime
emissao_co2['Date'] = pd.to_datetime(emissao_co2['Date'], 
                                     format='%d-%m-%Y', errors='coerce')

In [ ]:
# Calcula Ano atual
ano_atual = datetime.now().year

In [ ]:
# Verifica valores máximo e mínimo das datas. 
print('Maior Data:',emissao_co2['Date'].max())
print('Menor Data:',emissao_co2['Date'].min())

In [ ]:
# Foi identificado que os dados da base iniciam-se todos em 1990, dessa forma, tudo o que está antes, será descartado.
datas_fora_intervalo = emissao_co2[
    (emissao_co2["Date"].dt.year < 1990) | 
    (emissao_co2["Date"].dt.year > 2025)
]
print(datas_fora_intervalo)
#print(len(datas_fora_intervalo))

In [ ]:
# Filtrar entre 1990 e o presente ano
emissao_co2 = emissao_co2[(emissao_co2['Date'].dt.year >= 1990) & 
                          (emissao_co2['Date'].dt.year <= ano_atual)]


In [ ]:
print(emissao_co2['Date'].dt.year.unique())


In [ ]:
# dataset com datas ajustadas no padrão datetime
emissao_co2.head()

In [ ]:
# Agora o dataset possui valores apenas entre 1990 e 2019
print('Maior Data:',emissao_co2['Date'].max())
print('Menor Data:',emissao_co2['Date'].min())

In [ ]:
# Verifica datas fora do padrão e formatação
datas_invalidas = emissao_co2[emissao_co2["Date"].isna()]
print(f"{len(datas_invalidas)} datas inválidas encontradas.")


### Padronização dos Países
##### Existe uma série de dados faltantes na coluna de Países. Nesta seção, isso será corrigido


In [ ]:
# Existe uma série de dados faltantes na coluna de Países
emissao_co2.head()

In [ ]:
""" 
Primeiro ordena pelas regiões (Region) em ordem alfabética

Dentro de cada região, ordena os países (Country)

Dentro de cada país, ordena cronologicamente pelas datas (Date)

Para visualizar os dados de forma mais organizada

Para facilitar análise por região/país/tempo

Para aplicar funções como .groupby() ou .rolling() de forma mais controlada
"""

# Ordena a base para facilitar preenchimento
emissao_co2 = emissao_co2.sort_values(by=["Region", "Country", "Date"])
emissao_co2.head()

In [ ]:
""" 
emissao_co2.groupby("Region")
Isso agrupa o DataFrame por "Region".
Cada grupo contém apenas as linhas de uma determinada região (como "Africa", "Asia", etc).

["Country"]
Seleciona apenas a coluna "Country" dentro de cada grupo.
Ou seja, vamos trabalhar só com os países de cada região separadamente.

.transform(...)
Aplica uma função linha por linha, preservando o tamanho original da coluna.
Ou seja, se você tem 100 linhas, continua com 100 linhas — só que com os dados transformados.

lambda x: x.ffill().bfill()
Aqui está a função que será aplicada a cada grupo (cada região):

x.ffill(): forward fill
Preenche valores nulos com o último valor válido acima.

x.bfill(): backward fill
Preenche valores nulos com o próximo valor válido abaixo (útil se o valor ausente estiver na primeira linha).

Juntos: ffill().bfill()
Essa combinação garante que todos os NaN serão preenchidos, seja com o valor anterior, seja com o próximo.

"""
# Preenche usando forward fill e back fill dentro de cada região
# Esse método assume que os países aparecem em blocos no mesmo grupo de região
emissao_co2["Country"] = emissao_co2.groupby("Region")["Country"].transform(lambda x: x.ffill().bfill())

In [ ]:
# Mostra linhas que ainda não têm país
print(emissao_co2[emissao_co2["Country"].isna()])

In [ ]:
#  Verificando valores faltantes, 
# Agora, somente a coluna "Kilotons of Co2" possui valores faltantes
emissao_co2.isnull().sum()

### Ajuste para Kilotons of Co2

##### Temos uma série de linhas com valores inexistentes. 

##### Para resolver isso, temos algumas abordagens.

##### - A primeira delas, é excluir todas as linhas com dados faltantes e seguir a análise. Ela é mais rápida, mas é possível que seja perdido dados valiosos. 

##### - A segunda, é utilizar uma base de dados contendo a população mundial de 1960-2016, proveniente do Kaggle. Dessa forma, iremos utilizar esta base para calcular a coluna de kilotons of co2 para cada país, no respectivo ano em que não há dados. Como essa base possui dados até o ano de 2016, ainda faltariam 3 anos para completar a análise. Dessa forma, vamos calcular os anos faltantes por meio de uma regressão linear. A partir disso, utilizando os valores de Metric Tons Per Capita e a população mundial será calculado o valor de Kilotons of Co2 por ano.

##### Calcular Kilotons of CO2 com base na população e nas emissões per capita

$$
Kilotons = \frac{População \times Metric\ Tons\ Per\ Capita}{1000}
$$




In [ ]:
emissao_co2.head()

In [ ]:
# Verificação de linhas nulas para a coluna de Kilotons of Co2.
linhas_nulas = emissao_co2[emissao_co2["Kilotons of Co2"].isnull()]
print("Linhas com 'Kilotons of Co2' nulo:")
linhas_nulas

In [ ]:
# Verificando valores faltantes, 
# Agora, somente a coluna Kilotons of Co2 possui valores faltantes
emissao_co2.isnull().sum()

In [ ]:
# Os paises com dados faltantes são apenas os:
set(linhas_nulas['Country']) # Usa-se set para pegar apenas os valores únicos da lista.

In [ ]:
# Leitura da base de dados da população mundial
pop_mundial = pd.read_csv('WorldPopulation.csv')

pop_mundial.head()

In [ ]:
# Substituir o UK por United Kingdom
pop_mundial["Country"] = pop_mundial["Country"].replace("UK", "United Kingdom")

In [ ]:
# Buscar o match de paises com dados faltantes que estão no dataframe de população mundial
# .unique() talvez não seja necessário REVER
match = set(pop_mundial["Country"].unique()) & set(linhas_nulas['Country'])
match

In [ ]:
""" 
pop_mundial["Country"].isin(match)

pop_mundial["Country"] é a coluna "Country" do DataFrame pop_mundial.

.isin(match) cria uma máscara booleana (uma série de True/False) indicando para cada linha se o valor na coluna "Country" está dentro do conjunto match.

Ou seja, essa expressão vai ser True para os países que estão no conjunto match e False para os que não estão.

pop_mundial[pop_mundial["Country"].isin(match)]

Aqui estamos usando essa máscara booleana para filtrar as linhas do DataFrame pop_mundial.

Só ficam as linhas onde o país está em match.

.copy()

Cria uma cópia do DataFrame filtrado, para evitar problemas futuros de SettingWithCopyWarning do pandas, que pode acontecer quando você mexe diretamente numa fatia (slice) do DataFrame original.

pop_mundial = ...

Substitui o DataFrame original pop_mundial pela versão filtrada (só com países que estão em match).

Essa linha filtra o DataFrame pop_mundial para manter apenas as linhas cujo país está presente no conjunto match, e depois cria uma cópia desse subconjunto, salvando de volta em pop_mundial.
"""

pop_mundial = pop_mundial[pop_mundial["Country"].isin(match)].copy()

In [ ]:
pop_mundial.head()

In [ ]:
# Conversão da data e extração do ano
# Rever essa conversão, acredito que já foi feita anteriormente REVER
emissao_co2["Date"] = pd.to_datetime(emissao_co2["Date"], errors="coerce")
emissao_co2["Ano"] = emissao_co2["Date"].dt.year


In [ ]:
# Selecionar apenas anos até 2016 e Kilotons ausentes
faltantes = emissao_co2[(emissao_co2["Kilotons of Co2"].isna()) & (emissao_co2["Ano"] <= 2016)].copy()

In [ ]:
""" 
melt "derrete" várias colunas em duas colunas: uma com o nome original da coluna (que vira uma variável categórica, aqui o ano) e outra com os valores dessas colunas (a população).


id_vars=["Country"]
Diz que a coluna "Country" deve permanecer fixa, ou seja, não será "derretida".

value_vars=[str(ano) for ano in range(1960, 2017)]
Especifica quais colunas serão "derretidas" (transformadas em valores de uma nova coluna). Aqui está criando uma lista de strings de anos, de 1960 até 2016 (porque o range vai até 2017, excluindo o último). Essas colunas são as que contêm a população por ano.

var_name="Ano"
O nome da nova coluna que vai conter os nomes das colunas originais, ou seja, os anos.

value_name="Populacao"
O nome da nova coluna que vai conter os valores correspondentes (população).

DataFrame pop_long com 3 colunas:

"Country" — país

"Ano" — ano (antes string)

"Populacao" — população daquele país naquele ano
"""

# Transformar pop_mundial para formato longo (Ano, Populacao)
pop_long = pop_mundial.melt(
    id_vars=["Country"],
    value_vars=[str(ano) for ano in range(1960, 2017)],
    var_name="Ano",
    value_name="Populacao"
)
pop_long["Ano"] = pop_long["Ano"].astype(int) # Será que já não foi convertido antes? REVER




In [ ]:
""" 

emissao_co2.index.names
Retorna uma lista (ou tupla) com o(s) nome(s) das colunas que compõem o índice do DataFrame. Exemplo: Index(['Ano', 'Country'], dtype='object')

if "Ano" in emissao_co2.index.names:
Verifica se a coluna "Ano" está no índice do DataFrame.

emissao_co2.reset_index()
O método reset_index() "desfaz" o índice, transformando as colunas do índice em colunas normais do DataFrame, e recriando um índice padrão numérico (0, 1, 2,...).

Atribuição emissao_co2 = emissao_co2.reset_index()
Salva o DataFrame com índice resetado, para garantir que "Ano" agora é uma coluna normal, não parte do índice.

"""
# Garantir que 'Ano' seja uma coluna, não índice
if "Ano" in emissao_co2.index.names:
    emissao_co2 = emissao_co2.reset_index()

In [ ]:
""" 
how="left"
Tipo da junção: left join.
Isso significa que o resultado vai manter todas as linhas do DataFrame faltantes, e vai adicionar as colunas de pop_long onde houver correspondência em "Country" e "Ano".
Caso não haja correspondência, as colunas de pop_long serão preenchidas com NaN.
"""
# Juntar os faltantes com população
faltantes_merged = faltantes.merge(pop_long, on=["Country", "Ano"], how="left")

In [ ]:
# Calcular estimativa de Kilotons
faltantes_merged["Kilotons estimado"] = (
    faltantes_merged["Populacao"] * faltantes_merged["Metric Tons Per Capita"] / 1000
)


In [ ]:
# Verifica se o DataFrame emissao_co2 não possui a coluna "Ano". Isso é importante para evitar criar essa coluna duas vezes ou sobrescrever dados existentes.
# Criar a coluna 'Ano' se não existir. Cria essa nova coluna extraindo o ano da coluna "Date" (que deve ser do tipo datetime)
# Atualizar o dataframe original com os valores estimados
# Garantir que 'Ano' também está no emissões_co2 original
if "Ano" not in emissao_co2.columns:
    emissao_co2["Ano"] = emissao_co2["Date"].dt.year

In [ ]:
""" 
faltantes_merged é um DataFrame que contém registros com dados estimados.

.iterrows() permite iterar linha por linha.

row representa uma linha individual (como uma série), e idx é o índice da linha.

Aqui estamos procurando, no DataFrame emissao_co2, a linha que tem o mesmo país e o mesmo ano da linha atual de faltantes_merged.

O operador & combina as duas condições (país E ano iguais).


emissao_co2.loc[condicao, "Kilotons of Co2"] seleciona a célula correspondente à condição.

row["Kilotons estimado"] é o valor estimado que será usado para preencher.

O código substitui o valor faltante na coluna "Kilotons of Co2" com esse valor estimado.


"""
# Usar merge com índice para localizar e preencher
for idx, row in faltantes_merged.iterrows():
    condicao = (emissao_co2["Country"] == row["Country"]) & (emissao_co2["Ano"] == row["Ano"])
    emissao_co2.loc[condicao, "Kilotons of Co2"] = row["Kilotons estimado"]

print("Valores estimados preenchidos com sucesso!")

In [ ]:
# Verificando valores faltantes, 
# Agora, somente a coluna Kilotons of Co2 possui valores faltantes
emissao_co2.isnull().sum()

In [ ]:
emissao_co2.head()

In [ ]:
emissao_co2.tail(10)

In [ ]:
# Verificação de linhas nulas para a coluna de Kilotons of Co2. Agora apenas a partir de 2017.
linhas_nulas = emissao_co2[emissao_co2["Kilotons of Co2"].isnull()]
print("Linhas com 'Kilotons of Co2' nulo:")
linhas_nulas

In [ ]:
print('Maior Data:',linhas_nulas['Date'].max())
print('Menor Data:',linhas_nulas['Date'].min())

In [ ]:
# pop_long = pop_mundial.melt(
#     id_vars=['Country'],
#     value_vars=[str(ano) for ano in range(1960, 2017)],
#     var_name='Ano',
#     value_name='Populacao'
# )
# pop_long['Ano'] = pop_long['Ano'].astype(int)

# Essa linha tenta converter a coluna 'Populacao' para um tipo numérico (por exemplo, de object → float) sem copiar os dados.

# Isso é necessário porque, se a coluna estiver como object, a interpolação pode falhar ou se comportar de forma errada.
# Fazer a regressão linear para prever os dados de população e depois calcular o kiloton a partir deles.
pop_long['Populacao'] = pop_long['Populacao'].infer_objects(copy=False)

""" 
groupby('Country') → Agrupa os dados por país. Assim, a interpolação será feita individualmente por país.

group_keys=False → Evita que o índice do grupo seja adicionado no resultado.

.apply(lambda g: g.interpolate(...)) → Aplica a interpolação linear para preencher valores faltantes dentro de cada grupo.

interpolate(method='linear', limit_direction='forward') → Estima os valores ausentes com base em uma linha reta (valores anteriores e posteriores).

limit_direction='forward' → preenche valores faltantes olhando para frente (ex: preenche 2002 com base em 2001 e 2003).
"""
pop_long['Populacao'] = (
    pop_long.groupby('Country', group_keys=False)['Populacao']
    .apply(lambda g: g.interpolate(method='linear', limit_direction='forward'))
)

anos_prev = [2017, 2018, 2019]
paises = pop_long['Country'].unique()

rows_futuras = []

""" 
Para cada país no array paises (definido antes com pop_long['Country'].unique()), o código vai:

Selecionar os dados.

Treinar um modelo.

Fazer previsões para os anos 2017–2019.
"""
for pais in paises:
    # Seleciona os dados da população somente daquele país.
    dados_pais = pop_long[pop_long['Country'] == pais].sort_values('Ano')
    # Cria uma instância do modelo de regressão linear do sklearn.
    modelo = LinearRegression()
    
    # Usar apenas anos disponíveis
    #  vetor de anos (coluna Ano) transformado em matriz de uma coluna (necessário para sklearn).
    x = dados_pais['Ano'].values.reshape(-1, 1)
    # vetor de valores da população.
    y = dados_pais['Populacao'].values
    if len(x) > 1:  # evitar erro com país de poucos dados
        modelo.fit(x, y)
        for ano in anos_prev:
            """ 
            Para cada ano em anos_prev (2017, 2018, 2019):
            Previsão da população com o modelo.
            """
            y_pred = modelo.predict([[ano]])[0]
            # Adiciona um dicionário com o país, ano e valor estimado na lista rows_futuras.
            rows_futuras.append({'Country': pais, 'Ano': ano, 'Populacao': y_pred})

# Adiciona as previsões ao DataFrame
pop_prev = pd.DataFrame(rows_futuras)
pop_long = pd.concat([pop_long, pop_prev], ignore_index=True)

# left quer dizer: mantenha todas as linhas de emissao_co2, mesmo que não exista população correspondente no pop_long.
# Se não houver correspondência, os valores da população virão como NaN.
df_final = pd.merge(
    emissao_co2, pop_long,
    how='left',
    on=['Country', 'Ano']
)

filtro_nulo = df_final['Kilotons of Co2'].isna() & df_final['Populacao'].notna()
""" 
df_final.loc[filtro_nulo, 'Kilotons of Co2']
Você está selecionando as linhas que têm emissões faltantes mas população disponível (filtro_nulo) e a coluna "Kilotons of Co2". Vai preencher essas células.

Do lado direito da atribuição:

df_final.loc[filtro_nulo, 'Populacao']
Pega a população dessas mesmas linhas.

df_final.loc[filtro_nulo, 'Metric Tons Per Capita']
Pega a emissão per capita (em toneladas métricas) dessas linhas.

Multiplica a população pelo valor per capita para obter as emissões totais:
Divide por 1000 para converter toneladas métricas em kilotoneladas (kilotons), que é a unidade da coluna "Kilotons of Co2".

.astype(float)
Garante que o resultado seja do tipo float, para evitar problemas de tipo.

calculando o valor faltante de emissão total de CO₂ (kilotons) a partir da população multiplicada pela emissão per capita, convertendo a unidade para kilotons, e preenchendo esses valores no DataFrame.
"""
df_final.loc[filtro_nulo, 'Kilotons of Co2'] = (
    (df_final.loc[filtro_nulo, 'Populacao'] *
     df_final.loc[filtro_nulo, 'Metric Tons Per Capita'] / 1000).astype(float)
)

In [ ]:
print(df_final.isna().sum())

In [ ]:
df_final[df_final['Country']=='Austria']

In [ ]:
# Verificando valores faltantes, 
# Agora, somente a coluna Kilotons of Co2 possui valores faltantes
df_final.isnull().sum()

In [ ]:
emissao_co2 = df_final.copy()

### Validação dos Países
##### Essa é apenas uma etapa de validação, para que seja garantido que os países que foram preenchidos a partir do ffill e bfill estejam com a Região correta. Para isto, vamos utilizar um dataset obtido do Kaggle, com a lista de países por região.

In [ ]:
# Leitura da bases de Países por Continentes(Regiões)
lista_paises = pd.read_csv('Countries by continents.csv', delimiter=';')
lista_paises.head()

In [ ]:
emissao_co2.head(5)

In [ ]:
lista_paises.head(5)

In [ ]:
# Padronizar textos
emissao_co2["Country"] = emissao_co2["Country"].str.strip()
emissao_co2["Region"] = emissao_co2["Region"].str.strip()
lista_paises["Country"] = lista_paises["Country"].str.strip()
lista_paises["Region"] = lista_paises["Region"].str.strip()

# Merge apenas com países presentes em emissao_co2
comparacao = emissao_co2.merge(lista_paises, on="Country", how="left", suffixes=("_emissao", "_oficial"))

# Identifica se a região está correta
comparacao["Regiao_Correta"] = comparacao["Region_emissao"] == comparacao["Region_oficial"]

# Agrupa por país e mostrar apenas 1 linha por país
resultado = comparacao[["Country", "Region_emissao", "Region_oficial", "Regiao_Correta"]].drop_duplicates()

# Exibir resultado
print("Comparação das regiões apenas para países presentes em emissao_co2:")
resultado.sort_values("Country")

In [ ]:
# Filtra os países com região incorreta e region_oficial não nula
erros = resultado[
    (resultado["Regiao_Correta"] == False) &
    (resultado["Region_oficial"].notna())
]

print("Países com região incorreta (e que têm Region_oficial definida):")
print(erros.sort_values("Country"))

As regiões encontradas são região de fronteiras entre Ásia e Europa. Dessa forma, foi considerado o que o foi preenchido pelo ffill e bfill anteriormente.

### Análise Exploratória (EDA)

##### A seção seguinte, apresenta algumas análises exploratórias da base de Emissões de CO2

##### Os comentários estão abaixo de cada gráfico.

In [ ]:
print('Soma total da emissão de CO2 por região dentro do horizonte de análise:\n', emissao_co2.groupby('Region')['Kilotons of Co2'].sum())

In [ ]:
print('Média da emissão de CO2 por região dentro do horizonte de análise:\n', emissao_co2.groupby('Region')['Kilotons of Co2'].mean())

In [ ]:
# Estatísticas descritivas resumidas por regiao
emissao_co2.groupby('Region')['Kilotons of Co2'].describe()

In [ ]:
emissao_asia     = emissao_co2[emissao_co2["Region"] == 'Asia']
emissao_europa   = emissao_co2[emissao_co2["Region"] == 'Europa']
emissao_americas = emissao_co2[emissao_co2["Region"] == 'Americas']
emissao_oceanias = emissao_co2[emissao_co2["Region"] == 'Oceania']
emissao_africa   = emissao_co2[emissao_co2["Region"] == 'Africa']

In [ ]:
asia = emissao_asia.groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)

In [ ]:
""" asia = emissao_asia.groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)
china = emissao_asia[emissao_asia["Country"] == "China"].groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)
total_asia = (china/asia)*100
print('Asia',total_asia)

europa = emissao_europa.groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)
russia = emissao_europa[emissao_europa["Country"] == "Russia"].groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)
total_europa = (russia/europa)*100
print('Europa:',total_europa)

america =emissao_americas.groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)
eua = emissao_americas[emissao_americas["Country"] == "United States"].groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)
total_america = (eua/america)*100
print('America:',total_america) """

# Essa célula foi comentada porque os cálculos acima foram transferidos para uma função

In [ ]:
def calcular_percentual_emissao(df, pais_alvo, nome_continente):
    total_por_regiao = df.groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)
    pais_por_regiao = df[df["Country"] == pais_alvo].groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)
    percentual = (pais_por_regiao / total_por_regiao) * 100

    print(f"{nome_continente}:\n{percentual}\n")
    
    return pais_por_regiao, total_por_regiao, percentual


china, asia,_ = calcular_percentual_emissao(emissao_asia, "China", "Ásia")
russia, europa,_ = calcular_percentual_emissao(emissao_europa, "Russia", "Europa")
eua, america,_ = calcular_percentual_emissao(emissao_americas, "United States", "América")

In [ ]:
# Valores absolutos (em Kilotons)
china_val = china.values[0]
asia_total = asia.values[0]
outros_asia = asia_total - china_val

russia_val = russia.values[0]
europa_total = europa.values[0]
outros_europa = europa_total - russia_val

eua_val = eua.values[0]
america_total = america.values[0]
outros_america = america_total - eua_val

# DataFrame com os dados
df = pd.DataFrame({
    'Região': ['Ásia', 'Europa', 'América'],
    'País': [china_val, russia_val, eua_val],
    'Outros': [outros_asia, outros_europa, outros_america]
})

# Plot stacked bar
plt.figure(figsize=(8, 5))
bar1 = plt.bar(df['Região'], df['País'], label='País selecionado', color=['red', 'blue', 'green'])
bar2 = plt.bar(df['Região'], df['Outros'], bottom=df['País'], label='Outros países', color='lightgray')

# Título e legendas
plt.title('Participação nas Emissões de CO₂ por Região (Kilotons)')
plt.ylabel('Emissões (Kilotons de CO₂)')
plt.legend()
plt.tight_layout()
plt.show()

A partir deste gráfico, é possível perceber a contribuição total de emissão dos top 3 países em suas respctivas regiões.

In [ ]:
# Agrupar por região e somar as emissões
emissoes_por_regiao = emissao_co2.groupby("Region")["Kilotons of Co2"].sum().sort_values(ascending=False)

# Plot
plt.figure(figsize=(8, 5))
emissoes_por_regiao.plot(kind="bar", color="skyblue", edgecolor="black")

plt.title("Emissões de CO₂ por Região (Kilotons)", fontsize=14)
plt.xlabel("Região")
plt.ylabel("Kilotons de CO₂")
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

Percebe-se que a região da Ásia é uma das que geram mais pegadas de CO2 em relação as demais regiões. A Oceania é a que menor gera pegadas de CO2.

In [ ]:
# Agrupar por país e somar as emissões
emissoes_por_pais_asia = emissao_asia.groupby("Country")["Kilotons of Co2"].sum().sort_values(ascending=True)

# Plot com barras horizontais
plt.figure(figsize=(8, 6))
emissoes_por_pais_asia.plot(kind="barh", color="skyblue", edgecolor="black")

plt.title("Emissões de CO₂ por País na Ásia (Kilotons)", fontsize=14)
plt.xlabel("Kilotons de CO₂")
plt.ylabel("País")
plt.tight_layout()
plt.show()

Dentro da região da Ásia, a China lidera as emissões de CO2.

In [ ]:
# Agrupar por região e somar as emissões
emissoes_por_regiao = emissao_co2.groupby("Region")["Kilotons of Co2"].mean().sort_values(ascending=False)

# Plot
plt.figure(figsize=(8, 5))
emissoes_por_regiao.plot(kind="bar", color="skyblue", edgecolor="black")

plt.title("Média Emissões de CO₂ por Região (Kilotons)", fontsize=14)
plt.xlabel("Região")
plt.ylabel("Kilotons de CO₂")
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [ ]:
# Garantir que a coluna 'Date' está em formato datetime
emissao_co2["Date"] = pd.to_datetime(emissao_co2["Date"])

# Criar uma nova coluna 'Ano'
emissao_co2["Ano"] = emissao_co2["Date"].dt.year

# Agrupar por ano e região
emissoes_ano_regiao = emissao_co2.groupby(["Ano", "Region"])["Kilotons of Co2"].sum().reset_index()

# Plotar um gráfico de linhas com uma linha para cada região
plt.figure(figsize=(10, 6))
for regiao in emissoes_ano_regiao["Region"].unique():
    df_r = emissoes_ano_regiao[emissoes_ano_regiao["Region"] == regiao]
    plt.plot(df_r["Ano"], df_r["Kilotons of Co2"], label=regiao)

plt.title("Emissões de CO₂ por Região ao Longo dos Anos")
plt.xlabel("Ano")
plt.ylabel("Kilotons de CO₂")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Nesse gráfico, é possível constatar a discrepância de emissões de CO2 da Ásia em relação aos demais Continentes.

In [ ]:
# Selecionar os N países com maior emissão total
top_paises = (
    emissao_co2.groupby("Country")["Kilotons of Co2"]
    .sum()
    .sort_values(ascending=False)
    .head(3)  # Altere esse número conforme necessário
    .index
)

# Filtrar apenas esses países no DataFrame
df_filtrado = emissao_co2[emissao_co2["Country"].isin(top_paises)]

# Plot
plt.figure(figsize=(10, 6))
for pais in df_filtrado["Country"].unique():
    df_p = df_filtrado[df_filtrado["Country"] == pais]
    plt.plot(df_p["Ano"], df_p["Kilotons of Co2"], label=pais)

plt.title("Emissões de CO₂ dos 3 Maiores Países ao Longo dos Anos")
plt.xlabel("Ano")
plt.ylabel("Kilotons de CO₂")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Dentre o Top 3, a China desponta como o maior emissor de CO2 ao longo do tempo. Esse crescimento acelerado pode ser também explicado pelo acelerado crescimento populacional do país

In [ ]:
# Selecionar os N países com maior emissão total
top_paises = (
    emissao_co2.groupby("Country")["Kilotons of Co2"]
    .sum()
    .sort_values(ascending=False)
    .head(10)  # Altere esse número conforme necessário
    .index
)

# Filtrar apenas esses países no DataFrame
df_filtrado = emissao_co2[emissao_co2["Country"].isin(top_paises)]

# Plot
plt.figure(figsize=(10, 6))
for pais in df_filtrado["Country"].unique():
    df_p = df_filtrado[df_filtrado["Country"] == pais]
    plt.plot(df_p["Ano"], df_p["Kilotons of Co2"], label=pais)

plt.title("Emissões de CO₂ dos 10 Maiores Países ao Longo dos Anos")
plt.xlabel("Ano")
plt.ylabel("Kilotons de CO₂")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Filtrar dados da Austrália ou outro país se desejar
df_china = emissao_co2[emissao_co2["Country"] == "China"]

# Plotar a correlação
plt.figure(figsize=(8, 6))
plt.scatter(df_china["Kilotons of Co2"], df_china["Metric Tons Per Capita"], color="orange", edgecolor="black")

plt.title("Correlação: Kilotons vs. Metric Tons Per Capita (China)")
plt.xlabel("Kilotons de CO₂")
plt.ylabel("Toneladas Métricas Per Capita")
plt.grid(True)
plt.tight_layout()
plt.show()

Percebe-se neste gráfico, uma correlação positiva forte, para a China por exemplo, que é o maior emisssor. Se fôssemos calcular o coeficiente de pearson, estaria próximo a 1. O que nos indica que quanto maior a tonelada métrica per capita, maior a emissão em kilotons de CO2.

In [ ]:
# Garantir que a coluna 'Date' está no formato datetime e criar coluna 'Ano'
emissao_co2["Date"] = pd.to_datetime(emissao_co2["Date"])
emissao_co2["Ano"] = emissao_co2["Date"].dt.year

# Filtrar para a Austrália (ou outro país)
df_china = emissao_co2[emissao_co2["Country"] == "China"]

# Plot com dois eixos y
fig, ax1 = plt.subplots(figsize=(10, 6))

# Primeiro eixo (Kilotons of CO2)
ax1.set_title("Emissões da China ao Longo dos Anos")
ax1.set_xlabel("Ano")
ax1.set_ylabel("Kilotons de CO₂", color="blue")
ax1.plot(df_china["Ano"], df_china["Kilotons of Co2"], color="blue", marker="o", label="Kilotons CO₂")
ax1.tick_params(axis="y", labelcolor="blue")

# Segundo eixo (Metric Tons per Capita)
ax2 = ax1.twinx()
ax2.set_ylabel("Toneladas Métricas Per Capita", color="green")
ax2.plot(df_china["Ano"], df_china["Metric Tons Per Capita"], color="green", marker="s", label="Toneladas per Capita")
ax2.tick_params(axis="y", labelcolor="green")

plt.grid(True)
fig.tight_layout()
plt.show()

A partir desse gráfico, nota-se o crescimento acelerado de emissões da China, fato que pode ser corroborado com a alta industrialização do país. Assim como a elevada taxa de crescimento da população, e consequentemente a concentração de emissões per capita

In [ ]:
# Garantir que não há valores nulos em 'Kilotons of Co2' e 'Region'
df_plot = emissao_co2.dropna(subset=["Kilotons of Co2", "Region"])

# Criar o boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x="Region", y="Kilotons of Co2", data=df_plot, palette="Set3")

plt.title("Distribuição de Emissões de CO₂ por Região")
plt.xlabel("Região")
plt.ylabel("Kilotons de CO₂")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Neste boxplot, fica claro a discrepância de emissões do continente asiático, seguido pelo americano e o europeu. Os valores do continente asiático chegam a ser outliers quando comparado com os demais.